In [1]:
import pymongo
from pymongo import MongoClient
import numpy as np
import pandas as pd


client = MongoClient()
db = client.obligations
collection = db.obligation_tracker
c = collection.find({ 'DESCRIPTION': { '$exists': True}})
obligations = []
responsbility = -1



for a in c:
    if a['RESPONSIBILITY'] in ["Supplier", "Customer" ]:
        obligations += [{'description': a['DESCRIPTION'], 'responsibility': 0, 'supplier':1 }]
    elif a['RESPONSIBILITY'] in ["Bloomberg", "Client", "UCLH", "Vodafone"]:
        obligations += [{'description': a['DESCRIPTION'], 'responsibility': 1, 'supplier':0 }]
    elif a['RESPONSIBILITY'] in ["Both - Bloomberg & Supplier", "Both - Client & Supplier", "Both - UCLH & Supplier", "Both - UCLH & Customer", "Both - Vodafone & Supplier"]:
        obligations += [{'description': a['DESCRIPTION'], 'responsibility': 2, 'supplier':0 }]
    else:
        print(a['RESPONSIBILITY'])
    
len(obligations)
df = pd.DataFrame(obligations)




-









-
-
-
-
-
-
-
-
-
-
-




In [2]:
df.shape

(9548, 3)

In [3]:
df

,description,responsibility,supplier
0,Monthly Productivity report showing total prod...,0,1
1,Quarterly Productivity report showing total pr...,0,1
2,Monthly Productivity report showing total prod...,0,1
3,Any renewal or extension of this Statement of ...,1,0
4,In the event of consecutive Service Level fail...,0,1
5,"Test results per exercise with scope, objectiv...",0,1
6,Details the results of each root-cause analysi...,0,1
7,Quarterly copies of all Service Provider train...,0,1
8,Annual Quality report showing overall quality ...,0,1
9,Annual Productivity report showing total produ...,0,1


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform()

In [6]:
X_train_counts[0].shape

(1, 9821)

In [7]:
X_train_counts.shape

(9548, 9821)

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(9548, 9821)

In [37]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])
text_clf = text_clf.fit(df[df['supplier']==0].iloc[1::2, :]['description'], df[df['supplier']==0].iloc[1::2, :]['responsibility'])
import numpy as np
predicted = text_clf.predict(df[df['supplier']==0].iloc[::2, :]['description'])
np.mean(predicted == df[df['supplier']==0].iloc[::2, :]['responsibility'])

0.80470162748643759

In [39]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),
 ])
_ = text_clf_svm.fit(df[df['supplier']==0].iloc[1::2, :]['description'], df[df['supplier']==0].iloc[1::2, :]['responsibility'])
predicted_svm = text_clf_svm.predict(df[df['supplier']==0].iloc[::2, :]['description'])
np.mean(predicted_svm == df[df['supplier']==0].iloc[::2, :]['responsibility'])

/Users/khaleeque.ansari/Clients/Khaleeque/DataSciencePractise/venv/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.83996383363471971

In [26]:
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

In [27]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-3),
}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(df.iloc[1::2, :]['description'], df.iloc[1::2, :]['supplier'])
gs_clf_svm.best_score_

0.80016757436112274

In [28]:
df

,description,responsibility,supplier
0,Monthly Productivity report showing total prod...,0,1
1,Quarterly Productivity report showing total pr...,0,1
2,Monthly Productivity report showing total prod...,0,1
3,Any renewal or extension of this Statement of ...,1,0
4,In the event of consecutive Service Level fail...,0,1
5,"Test results per exercise with scope, objectiv...",0,1
6,Details the results of each root-cause analysi...,0,1
7,Quarterly copies of all Service Provider train...,0,1
8,Annual Quality report showing overall quality ...,0,1
9,Annual Productivity report showing total produ...,0,1


In [29]:
df.size

28644

In [30]:
28644/4

7161.0

In [31]:
df['supplier'].size

9548

In [32]:
28644/3

9548.0

In [34]:
df[df['supplier']==0]['description']

3       Any renewal or extension of this Statement of ...
17      Holiday dates may vary each year. Bloomberg an...
18      In addition to the termination rights provided...
21      As set forth in Section 2.07 of the Master Agr...
38      Any renewal or extension of this Statement of ...
39      Holiday dates may vary each year. Bloomberg an...
40      In addition to the termination rights provided...
43      As set forth in Section 2.07 of the Master Agr...
59      Any renewal or extension of this Statement of ...
85      Bloomberg has the right to adjust the number o...
88      As set forth in Section 2.07 of the Master Agr...
89      Bloomberg may also terminate this SOW in its d...
90      Any renewal or extension of this Statement of ...
91      Holiday dates may vary each year. Bloomberg an...
95      As set forth in Section 2.07 of the Master Agr...
111     Any renewal or extension of this Statement of ...
112     Holiday dates may vary each year. Bloomberg an...
113     In add

In [35]:
df[df['supplier']==0]['responsibility']

3       1
17      2
18      1
21      1
38      2
39      2
40      1
43      1
59      2
85      1
88      1
89      1
90      2
91      2
95      1
111     2
112     2
113     1
116     1
132     2
133     2
134     1
137     1
153     2
154     2
155     1
158     1
174     2
175     2
176     1
       ..
9416    2
9425    2
9426    2
9428    2
9448    1
9461    1
9464    2
9465    2
9475    1
9477    2
9479    2
9480    2
9490    1
9491    2
9493    2
9494    2
9503    1
9505    2
9507    2
9508    2
9517    1
9519    2
9521    2
9522    2
9531    1
9533    2
9535    2
9536    2
9545    1
9547    2
Name: responsibility, Length: 2211, dtype: int64

In [40]:
test = ["51.2 If the approval of the Secretary of State is not given to any of the Broxbourne Proposal, Cambridge Proposal, Cheshunt Proposal, Harlow Proposal or Southend Victoria Proposal pursuant to paragraph 51.1 the Secretary of State may require the Franchisee to develop a detailed design for the redevelopment of an alternative Station (“Alternative Redevelopment Station”) and suggest a timeframe for such redevelopment work, such detailed design and suggested timeframe to be subject to the approval of the Secretary of State. If the Secretary of State requires the Franchisee to provide such detailed design and suggested timeframe then the Franchisee shall do so within ninety (90) days of such request. 51.3 The Franchisee shall implement the Approved Detailed Design for each of the Redevelopment Stations in order that the redevelopment work is completed in accordance with the Approved Detailed Design by no later than: (c) the date approved by the Secretary of State pursuant to paragraph 51.2 in the case of any Alternative Redevelopment Station.",
"1.1 (a) The Parties shall hold a Franchise Performance Meeting at least once in every Reporting Period (or such other interval as the Secretary of State may notify to the Franchisee in writing) at a time and location notified to the Franchisee by the Secretary of State. 8.1 The Franchisee shall (including in accordance with the requirements of paragraph 5.1 of Part 2 (Special Terms related to Committed Obligations) of Schedule 6.2 (Committed Obligations)) prepare and submit to the Secretary of State a periodic report in each Reporting Period containing such information as the Secretary of State may reasonably specify on or after commencement of this Agreement or from time to time in accordance with paragraph 8.2 below for the previous quarter, or such other period as may be reasonably required and disaggregated to the extent that the Secretary of State shall require. 9.1 The Franchisee shall prepare and at all times during the Franchise Term maintain true, up to date and complete accounting records as are required to be kept under Section 386 of the Companies Act 2006. Such records shall be prepared on a consistent basis for each Reporting Period. 9.2(a) The Franchisee shall deliver to the Secretary of State, within ten (10) Weekdays of the end of each Reporting Period: (i) Management Accounts for such Reporting Period, setting out a cashflow statement, profit and loss account and balance sheet for that Reporting Period and cumulatively for the Franchisee Year to date; (ii) Written confirmation that the Management Accounts, to the best of the knowledge, information and belief of the board of directors of the Franchisee, contain a true and accurate reflection of the current assets and liabilities of the Franchisee (including contingent assets or liabilities and known business risks and opportunities) and, to the extent that they do not, identify in a written report relevant issues in reasonable detail and provide such further information that the Secretary of State shall reasonably require in relation; and (iii) In circumstances where the Franchisee was in a Lock-up Period during such Reporting Period, written confirmation from a statutory director of the Franchisee that the Franchisee has complied with the restrictions applicable during a Lock-up Period pursuant to paragraph 3 of Schedule 12 (Financial Covenants and Bonds). 9.2(b)(vi) where the level of financial performance reported in the Management Accounts is, in the reasonable opinion of the Secretary of State, materially worse than forecast by the Franchisee in its current Business Plan, the Secretary of State may require the Franchisee to prepare and submit to him, as soon as reasonably practicable, a Financial Action Plan to ensure that the level of financial performance forecast in its current Business Plan for the remainder of the currency of that Business Plan is achieved and the Franchisee shall use all reasonable endeavours to implement such Financial Action Plan 5. Review of Compliance 5.1 Progress with Committed Obligations shall be considered and discussed at Franchise Performance Meetings and the Franchisee shall ensure that progress with regard to Committed Obligations is included in Periodic Update Reports provided in accordance with paragraph 8 of Schedule 11.2 (Management Information). 5.2 In addition to its obligation under paragraph 5.1, the Franchisee shall from time to time promptly provide such evidence of its compliance with any Committed Obligation as the Secretary of State may reasonably request. 1.9 Sch 7.1: The Secretary of State may at any time after a Charging Review vary, on giving not less than three (3) months' notice in writing, any of the Cancellations Benchmarks, Annual Cancellations Benchmarks, TOC Minute Delay Benchmarks and/or the Annual TOC Minute Delay Benchmarks to reflect the Secretary of State's reasonable view of the performance trajectory set as part of such Charging Review. Where the Secretary of State exercises his right pursuant to this paragraph 1.9, the relevant Cancellations Benchmark Table, Annual Cancellations Benchmark Table, TOC Minute Delay Benchmark Table and/or Annual TOC Minute Delay Benchmark Table shall be deemed to have been amended accordingly. The exercise by the Secretary of State of his rights pursuant to this paragraph 1.9 shall be a Change as specified in paragraph (k) of the definition of Change. 79.6 Sch 6: The Franchisee shall prepare and submit a report to the Secretary of State in advance of each Franchise Performance Meeting (in such format as the Secretary of State may reasonably require) setting out: (a) the level of take-up and usage of smart ticketing schemes and ABT by users of the Passenger Services, for each completed Reporting Period during the Franchise Period; (b) the steps that the Franchisee is taking to increase take-up of smart ticketing schemes and ABT schemes operated by the Franchisee and increase usage of smart ticketing schemes by users of the Passenger Services, and the Franchisee shall present the report at the Franchise Performance Meeting. 76.9 Sch 6: The Franchisee shall prepare and submit a report to the Secretary of State in advance of each Franchise Performance Meeting (in such format the Secretary of State may reasonably require) setting out the level of take-up and usage of the Anytime Flex Carnet and the Off Peak Flex Carnet by users of the Passenger Services, for each completed Reporting Period during the Franchise Period and the Franchisee shall present the report at the Franchise Performance Meeting."]

In [41]:
test

['51.2 If the approval of the Secretary of State is not given to any of the Broxbourne Proposal, Cambridge Proposal, Cheshunt Proposal, Harlow Proposal or Southend Victoria Proposal pursuant to paragraph 51.1 the Secretary of State may require the Franchisee to develop a detailed design for the redevelopment of an alternative Station (“Alternative Redevelopment Station”) and suggest a timeframe for such redevelopment work, such detailed design and suggested timeframe to be subject to the approval of the Secretary of State. If the Secretary of State requires the Franchisee to provide such detailed design and suggested timeframe then the Franchisee shall do so within ninety (90) days of such request. 51.3 The Franchisee shall implement the Approved Detailed Design for each of the Redevelopment Stations in order that the redevelopment work is completed in accordance with the Approved Detailed Design by no later than: (c) the date approved by the Secretary of State pursuant to paragraph 51

In [42]:
len(test)

2

In [43]:
predicted_svm = text_clf_svm.predict(df[df['supplier']==0].iloc[::2, :]['description'])

In [44]:
predicted_svm

array([2, 1, 2, ..., 2, 2, 2])

In [45]:
predicted_svm = text_clf_svm.predict(test)

In [46]:
predicted_svm

array([1, 1])

In [49]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),
 ])
_ = text_clf_svm.fit(df[df['supplier']==0]['description'], df[df['supplier']==0]['responsibility'])
predicted_svm = text_clf_svm.predict(df['description'])
np.mean(predicted_svm == df['responsibility'])

/Users/khaleeque.ansari/Clients/Khaleeque/DataSciencePractise/venv/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.21512358609132803